# Feature Transformation with Amazon a SageMaker Processing Job and Scikit-Learn

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Scikit-Learn are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Scikit-Learn in a managed SageMaker environment to run our processing workload.

![](img/prepare_dataset.png)

![](img/processing.jpg)


## Contents

1. Setup Environment
1. Setup Input Data
1. Setup Output Data
1. Build a Spark container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Data

# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [1]:
!pip install boto3

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
from time import gmtime, strftime
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Setup Input Data

In [3]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/


In [4]:
!aws s3 ls $s3_input_data

2020-03-21 17:59:04   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-03-21 17:59:05   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


# Setup Output Data

In [5]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-scikit-processor-{}'.format(timestamp_prefix)
scikit_processing_job_name = 'amazon-reviews-scikit-processor-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(scikit_processing_job_name))

Processing job name:  amazon-reviews-scikit-processor-2020-03-22-21-04-18


# Run the Processing Job using Amazon SageMaker

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for processing in the job configuration.

Review the Spark processing script.

In [6]:
cat preprocess-scikit-text-to-bert.py

import argparse
import json
import os
import pandas as pd
import numpy as np
import csv
import glob
from pathlib import Path

# requirements.txt
import subprocess
import sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'])
import torch
import transformers as ppb
import sklearn
from sklearn.model_selection import train_test_split

def list_arg(raw_value):
    """argparse type for a list of strings"""
    return str(raw_value).split(',')


def parse_args():
    # Unlike SageMaker training jobs (which have `SM_HOSTS` and `SM_CURRENT_HOST` env vars), processing jobs to need to parse the resource config file directly
    resconfig = {}
    try:
        with open('/opt/ml/config/resourceconfig.json', 'r') as cfgfile:
            resconfig = json.load(cfgfile)
    except FileNotFoundError:
        print('/opt/ml/config/resourceconfig.json not found.  current_host is unknown.')
        pass # Ignore

    # Local testing with CLI args
    parser = argpar

Run this script as a processing job.  You specify the command (`/opt/program/submit` for this Spark processor.)  You also need to specify one `ProcessingInput` with the `source` argument of the Amazon S3 bucket and `destination` is where the script reads this data from `/opt/ml/processing/input` (inside the Docker container.)  All local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to.  For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`.  You also give the `ProcessingOutput` value for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The arguments parameter in the `run()` method are command-line arguments in our `preprocess-scikit-text-to-bert.py` script.

In [13]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor = SKLearnProcessor(framework_version='0.20.0',
                             role=role,
                             instance_type='ml.m5.4xlarge',
                             instance_count=2)

In [14]:
# ShardedS3Key to spread the transformations across all nodes
processor.run(code='preprocess-scikit-text-to-bert.py',
                      inputs=[ProcessingInput(source=s3_input_data,
                                              destination='/opt/ml/processing/input/data/',
                                              s3_data_distribution_type='ShardedByS3Key')],
                      outputs=[
                               ProcessingOutput(s3_upload_mode='EndOfJob',
                                                output_name='raw-labeled-split-balanced-header-train',
                                                source='/opt/ml/processing/output/raw/labeled/split/balanced/header/train'),
                               ProcessingOutput(s3_upload_mode='EndOfJob',
                                                output_name='raw-labeled-split-balanced-header-validation',
                                                source='/opt/ml/processing/output/raw/labeled/split/balanced/header/validation'),
                               ProcessingOutput(s3_upload_mode='EndOfJob',
                                                output_name='raw-labeled-split-balanced-header-test',
                                                source='/opt/ml/processing/output/raw/labeled/split/balanced/header/test'),
                      ],
                      logs=True,
                      wait=False)



Job Name:  sagemaker-scikit-learn-2020-03-22-21-16-53-826
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-22-21-16-53-826/input/code/preprocess-scikit-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'raw-labeled-split-balanced-header-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-22-21-16-53-826/output/raw-labeled-split-balanced-header-train', 'LocalPath': '/opt/ml/processing/output/raw/labeled/split/balanced/header/tr

In [15]:
scikit_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, scikit_processing_job_name)))


In [16]:
from IPython.core.display import display, HTML

# Our job writes to `processing_job_name` since we are using ProcessingOutput above
scikit_processing_job_s3_output_prefix = scikit_processing_job_name

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, scikit_processing_job_s3_output_prefix, region)))


# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [11]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=scikit_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-22-21-04-21-521/input/code/preprocess-scikit-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'raw-labeled-split-balanced-header-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-22-21-04-21-521/output/raw-labeled-split-balanced-header-train', 'LocalPath': '/opt/ml/processing/output/raw/labeled/split/balanced/header/train

# Inspect the Processed Output Data

## The next cells will not work properly until the job completes above.

Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [12]:
output_config = processing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'raw-labeled-split-balanced-header-train':
        processed_balanced_train_data = output['S3Output']['S3Uri']
    if output['OutputName'] == 'raw-labeled-split-balanced-header-validation':
        processed_balanced_validation_data = output['S3Output']['S3Uri']        
    if output['OutputName'] == 'raw-labeled-split-balanced-header-test':
        processed_balanced_test_data = output['S3Output']['S3Uri']
        
print(processed_balanced_train_data)
print(processed_balanced_validation_data)
print(processed_balanced_test_data)

s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-22-21-04-21-521/output/raw-labeled-split-balanced-header-train
s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-22-21-04-21-521/output/raw-labeled-split-balanced-header-validation
s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-03-22-21-04-21-521/output/raw-labeled-split-balanced-header-test


In [ ]:
!aws s3 ls $processed_balanced_train_data/

In [ ]:
!aws s3 ls $processed_balanced_validation_data/

In [ ]:
!aws s3 ls $processed_balanced_test_data/

# Pass `scikit_processing_job_s3_output_prefix` above as input to the next notebook

In [ ]:
print(scikit_processing_job_s3_output_prefix)

In [ ]:
%store scikit_processing_job_s3_output_prefix
